### Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через
аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта
(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
● Наименование вакансии.
● Предлагаемую зарплату (отдельно минимальную и максимальную).
● Ссылку на саму вакансию.
● Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение).
Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с
помощью dataFrame через pandas.

In [268]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pprint import pprint

In [305]:
def parser_hh(vacancy):

    vacancy_date = []
    
    params = {
        'text': vacancy,
        'area': '2',
        'clusters': 'true',
        'enable_snippets': 'true',
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }

    main_link = 'https://spb.hh.ru'

    html = requests.get(main_link + '/search/vacancy/', params=params, headers=headers)

    if html.ok:
        parsed_html = bs(html.text, 'lxml')

        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())

    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(main_link + '/search/vacancy/', params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text, 'lxml')
            vacancy_block = parsed_html.find('div', {'class': 'vacancy-serp'})
            vacancs_list = vacancy_block.findChildren(recursive = False)
                
            for item in vacancs_list:
                vacancy_date.append(parser_item_hh(item))
    
    return vacancy_date

In [383]:
def parser_item_hh(item):

    vacancy_date = {}
    
    tag_link = item.find('span', {'class': 'g-user-content'})
    if tag_link != None:
        tag_link_info = tag_link.findChild()
        vacancy_name = tag_link_info.getText()
        vacancy_link = tag_link_info['href']
        
        vacancy_date['vacancy_name'] = vacancy_name
        vacancy_date['vacancy_link'] = vacancy_link
              
    # company_name
    
        company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}).find('a').getText()
    
        vacancy_date['company_name'] = company_name
    
    # city
        city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText().split(', ')[0]
    
        vacancy_date['city'] = city
    
    #metro station
        metro_station = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).findChild()

        if not metro_station:
            metro_station = None
        else:
            metro_station = metro_station.getText()
        
        vacancy_date['metro_station'] = metro_station
    
    #salary
        salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        if not salary:
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            salary = salary.getText().replace(u'\xa0', u'')
        
            salary = re.split(r'\s|-', salary)
        
            if salary[0] == 'до':
                salary_min = None
                salary_max = int(salary[1])
            elif salary[0] == 'от':
                salary_min = int(salary[1])
                salary_max = None
            else:
                salary_min = int(salary[0])
                salary_max = int(salary[1])            
        
            salary_currency = salary[2]
        
        vacancy_date['salary_min'] = salary_min
        vacancy_date['salary_max'] = salary_max
        vacancy_date['salary_currency'] = salary_currency
    
    # site
        vacancy_date['site'] = 'https://spb.hh.ru'
    
    return vacancy_date

In [384]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(parser_hh(vacancy))
    #vacancy_date.extend(_parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)
    df_final = df.dropna(axis='index', how='any', subset=['vacancy_name'])

    return df_final

In [385]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

In [386]:
df

,vacancy_name,vacancy_link,company_name,city,metro_station,salary_min,salary_max,salary_currency,site
0,Data Scientist,https://spb.hh.ru/vacancy/37369353?query=Python,ЦРТ | Группа компаний,Санкт-Петербург,Выборгская,NaN,NaN,None,https://spb.hh.ru
1,Web-аналитик,https://spb.hh.ru/vacancy/37163724?query=Python,"Риалвеб, Интернет-агентство",Санкт-Петербург,None,NaN,NaN,None,https://spb.hh.ru
3,Marketing Analyst,https://spb.hh.ru/vacancy/37361879?query=Python,PRISMA,Санкт-Петербург,Василеостровская,NaN,NaN,None,https://spb.hh.ru
4,Разработчик DWH [id 21076],https://spb.hh.ru/vacancy/36308035?query=Python,ПАО «Газпром нефть»,Санкт-Петербург,Площадь Восстания,NaN,NaN,None,https://spb.hh.ru
5,Python Developer (Django),https://spb.hh.ru/vacancy/37106283?query=Python,Muzlab,Санкт-Петербург,Приморская,100000.0,NaN,руб.,https://spb.hh.ru
...,...,...,...,...,...,...,...,...,...
1041,Инженер-программист,https://spb.hh.ru/vacancy/37084244?query=Python,ФГБУ Национальный Медицинский Исследовательск...,Санкт-Петербург,None,NaN,NaN,None,https://spb.hh.ru
1042,Middle/Senior Python Backend Developer (Remote...,https://spb.hh.ru/vacancy/36749862?query=Python,ООО АппФоллоу,Санкт-Петербург,None,NaN,NaN,None,https://spb.hh.ru
1043,Программист PHP,https://spb.hh.ru/vacancy/36975765?query=Python,ООО Арго Медиа,Санкт-Петербург,Политехническая,70000.0,120000.0,руб.,https://spb.hh.ru
1045,ASR Разработчик (Центр искусственного интеллекта),https://spb.hh.ru/vacancy/36936266?query=Python,"ПАО «МТС»,IT",Санкт-Петербург,None,NaN,NaN,None,https://spb.hh.ru


In [609]:
def parser_superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy,
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }
    
    main_link = 'https://russia.superjob.ru/vacancy/search/'
       
    html = requests.get(main_link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'lxml')
    
        page_block = parsed_html.find('div', {'class': "_3zucV L1p51 undefined _2guZ- _GJem"})
    if not page_block:
        last_page = 0
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page + 1
        html = requests.get(main_link, params=params, headers=headers)

        if html.ok:
            parsed_html = bs(html.text, 'lxml')
            vacancy_items = parsed_html.find_all('div', {'class': 'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})
                        
            for item in vacancy_items:
                vacancy_date.append(parser_item_superjob(item))
                
    return vacancy_date

In [610]:
def parser_item_superjob(item):

    vacancy_date = {}
    
    # vacancy_name
    vacancy_name = item.find('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'}).text
    vacancy_date['vacancy_name'] = vacancy_name
    
    #vacancy_link
    vacancy_link = item.find('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'}).find('a')['href']
    vacancy_date['vacancy_link'] = 'https://russia.superjob.ru' + vacancy_link
    
    # company_name
    location = item.find('div', {'class': '_3_eyK _3P0J7 _9_FPy'})
    company_name = location.find('a')
    if not company_name:
        company_name = None
    else:
        company_name = company_name.text
    vacancy_date['company_name'] = company_name
    
    # city
    city = location.find('span', {'class': '_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'}).text
    city = re.split(r'• |,', city)
    vacancy_date['city'] = city[1]
   
    #metro station
    if len(city) > 2:
        metro_station = city[2]
    else:
        metro_station = None
    vacancy_date['metro_station'] = metro_station
    
    #salary
    salary = item.find('span', {'class': '_3mfro _2Wp8I _1qw9T f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.text.replace(u'&nbsp;', u'')
        salary = re.split(r'\s|-', salary)
        if salary[0] == 'По':
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            if salary[0] == 'до':
                salary_min = None
                salary_max = int(salary[1] + salary[2])
                salary_currency = salary[3]
            elif salary[0] == 'от':
                salary_min = int(salary[1] + salary[2])
                salary_max = None
                salary_currency = salary[3]
            else:
                salary_min = int(salary[0] + salary[1])
                salary_max = int(salary[3] + salary[4])            
                salary_currency = salary[5]
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency

    # site
    vacancy_date['site'] = 'https://russia.superjob.ru'
    
    return vacancy_date

In [611]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)
    df_final = df.dropna(axis='index', how='any', subset=['vacancy_name'])

    return df_final

In [612]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

In [613]:
df

,vacancy_name,vacancy_link,company_name,city,metro_station,salary_min,salary_max,salary_currency,site
0,Разработчик Python,https://russia.superjob.ru/vakansii/razrabotch...,Технопарк «Сколково»,Москва,None,NaN,NaN,None,https://russia.superjob.ru
1,Разработчик приложений (Python),https://russia.superjob.ru/vakansii/razrabotch...,ДЕНЬГИ СРАЗУ,Москва,None,NaN,150000.0,руб.,https://russia.superjob.ru
2,Python / Django Developer (Middle),https://russia.superjob.ru/vakansii/python-339...,Effective,Воронеж,None,40000.0,80000.0,руб.,https://russia.superjob.ru
3,Разработчик Python,https://russia.superjob.ru/vakansii/razrabotch...,ADV,Москва,None,NaN,NaN,None,https://russia.superjob.ru
4,Преподаватель по курсу (Python для машинного о...,https://russia.superjob.ru/vakansii/prepodavat...,ДПО Центр «Профессионал»,Москва,None,NaN,NaN,None,https://russia.superjob.ru
...,...,...,...,...,...,...,...,...,...
71,Учитель робототехники,https://russia.superjob.ru/vakansii/uchitel-ro...,ГБОУ Школа № 1561,Москва,None,NaN,NaN,None,https://russia.superjob.ru
72,Rust Developer,https://russia.superjob.ru/vakansii/rust-devel...,TR Logic LLC,Москва,None,160000.0,164000.0,руб.,https://russia.superjob.ru
73,Junior QA Engineer for IoT Project,https://russia.superjob.ru/vakansii/junior-qa-...,ХайКво Солюшенс,Санкт-Петербург,None,NaN,NaN,None,https://russia.superjob.ru
74,Инженер-программист,https://russia.superjob.ru/vakansii/inzhener-p...,Академия цифровых технологий,Санкт-Петербург,None,24000.0,NaN,руб.,https://russia.superjob.ru
